# Software and Systems Performance - Project 
Coverage Analysis of YCSB benchmark tool for Mongodb Database

In [1]:
import matplotlib.pyplot as plt
import json

## Overall line coverage
<img src="ss.png"/>

In [2]:
with open('json_outputs_with_branch/mongod_main.gcov.json' ,'r') as f:
    data=json.load(f)
    data=data['files']
    print(data[0].keys())

dict_keys(['lines', 'functions', 'file'])


In [3]:
def gen_demangled_index(data):
    demangled_index={}
    for function in data['functions']:
        demangled_index[function['name']]=function['demangled_name']
    return demangled_index
demangled_index=(gen_demangled_index(data[0]))


In [4]:
data[0]['lines'][0]

{'branches': [],
 'count': 6,
 'line_number': 265,
 'unexecuted_block': False,
 'function_name': '_ZN5mongo12_GLOBAL__N_110logStartupEPNS_16OperationContextE'}

#### data[i]['lines'] has the data of every line of file i. 

In [5]:
def get_line_coverage(data):
    cnt=0
    for file in data:
        if file['file'].endswith('.cpp'):
            for line in file['lines']:
                if(line['count']>0):
                    cnt+=1
            return (cnt,len(file['lines']))
get_line_coverage(data)

(455, 728)

In [6]:

h_files=0
cpp_files=0
for file in data:
    if file['file'].endswith('.h') or file['file'].endswith('.hpp'):
        h_files+=1
    else:
        cpp_files+=1
        print(file['file'])

print(h_files,cpp_files)

src/mongo/db/mongod_main.cpp
279 1


In [10]:
# Get the cpp file data and ignore the .h file data

CPP_FILE=None
for file in data:
    if file['file'].endswith('.h') or file['file'].endswith('.hpp'):
        continue
    else:
        print(file['file'])
        CPP_FILE  = file


src/mongo/db/mongod_main.cpp


The number of cpp files in a gcov file is just 1 while the remaining files in the data are all header files. We will find line coverage only for the cpp files

In [8]:
def get_function_coverage(data):
    cnt=0
    for file in data:
        if file['file'].endswith('.cpp'):
            for function in file['functions']:
                if(function['execution_count']>0):
                    cnt+=1
            return (cnt,len(file['functions']))
get_function_coverage(data)


(19, 31)

#### To find the top 3 lengthiest unexecuted functions in the .cpp file


In [10]:
def get_longest_unused_functions(data):
    unex_func=[]
    for file in data:
        if file['file'].endswith('.cpp'):
            for function in file['functions']:
                if(function['execution_count']==0):
                    unex_func.append((function['demangled_name'],function['end_line']-function['start_line']+1))
            unex_func=sorted(unex_func,key=lambda x:-x[1])
            return unex_func[:3]
get_longest_unused_functions(data)

[('mongo::(anonymous namespace)::shutdownProcessByDBPathPidFile(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&)',
  55),
 ('mongo::(anonymous namespace)::makeReplicaSetNodeExecutor(mongo::ServiceContext*)',
  15),
 ('mongo::(anonymous namespace)::initializeCommandHooks(mongo::ServiceContext*)::MongodCommandInvocationHooks::onAfterAsyncRun(std::shared_ptr<mongo::RequestExecutionContext>, mongo::CommandInvocation*)',
  5)]

### Branch Coverage 

#### Sample Branch JSON
It has 3 fields:
* fallthrough: 
    * True - Control moves to next line after executing the branch
    * False - Control jumps to another line (Branch taken)
* Count: Number of times the branch condition was evaluated
* Throw: If branch throws an exception 

In [14]:
Flag = False
for line in CPP_FILE['lines']:
    for branch in line['branches']:
        Flag = True
        print(branch)
        break
    if Flag:
        break

{'fallthrough': True, 'count': 6, 'throw': False}


In [ ]:
# A branch must have nonzero counts for both fallthrough=True and fallthrough=False to have 100% coverage
def get_branch_coverage(file):
    NUM_BRANCHES = 0
    BRANCHES_TAKEN = 0
    for line in file['lines']:
        if not line['branches']:
            continue
        
        NUM_BRANCHES += 2
        branches_taken = {
            "fallthrough": False,
            "jump": False
        }
        for branch in line['branches']:
            if branch['fallthrough'] and branch['count'] != 0:
                branches_taken['fallthrough'] = True
            
            elif (not branch['fallthrough']) and branch['count'] != 0:
                branches_taken['jump'] = True

        if branches_taken['fallthrough']:
            BRANCHES_TAKEN += 1
        if branches_taken['jump']:
            BRANCHES_TAKEN += 1
    
    return BRANCHES_TAKEN, NUM_BRANCHES
        